In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'SVHN.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_labels = save['train_labels']
    test_data = save['test_dataset']
    test_labels = save['test_labels']
    valid_data = save['valid_dataset']
    valid_labels = save['valid_labels']
    del save
    

pickle_file = 'SVHN1.pickle'
with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_data_1 = save['train_dataset1']

pickle_file = 'SVHN2.pickle'
with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_data_2 = save['train_dataset2']
    del save  # hint to help gc free up memory

pickle_file = 'SVHN3.pickle'
with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_data_3 = save['train_dataset3']
    del save  # hint to help gc free up memory
    
train_data = np.concatenate((train_data_1, train_data_2, train_data_3), axis=0)

In [3]:
del train_data_1
del train_data_2
del train_data_3 
print('Validation set', valid_data.shape, valid_labels.shape)
print('Test set', test_data.shape, test_labels.shape)
print('Training set', train_data.shape, train_labels.shape)

Validation set (6000, 32, 32) (6000,)
Test set (26032, 32, 32) (26032,)
Training set (598388, 32, 32) (598388,)


In [4]:
image_size = 32
num_labels = 10
num_channels = 1 # grayscale

def reformat(data, labels):
    reformatted_data = data.reshape((-1, image_size, image_size, num_channels)).astype(np.float32)
    reformatted_labels = labels.astype(np.int32)
    return reformatted_data, reformatted_labels

In [5]:
train_data, train_labels = reformat(train_data, train_labels)
test_data, test_labels = reformat(test_data, test_labels)
valid_data, valid_labels = reformat(valid_data, valid_labels)
print('Training set', train_data.shape, train_labels.shape)
print('Test set', test_data.shape, test_labels.shape)
print('Validation set', valid_data.shape, valid_labels.shape)

Training set (598388, 32, 32, 1) (598388,)
Test set (26032, 32, 32, 1) (26032,)
Validation set (6000, 32, 32, 1) (6000,)


In [6]:
def LecunLCN(X, image_shape, threshold=1e-4, radius=7, use_divisor=True):
    """Local Contrast Normalization"""
    """[http://yann.lecun.com/exdb/publis/pdf/jarrett-iccv-09.pdf]"""

    # Get Gaussian filter
    filter_shape = (radius, radius, image_shape[3], 1)

    #self.filters = theano.shared(self.gaussian_filter(filter_shape), borrow=True)
    filters = gaussian_filter(filter_shape)
    X = tf.convert_to_tensor(X, dtype=tf.float32)
    # Compute the Guassian weighted average by means of convolution
    convout = tf.nn.conv2d(X, filters, [1,1,1,1], 'SAME')

    # Subtractive step
    mid = int(np.floor(filter_shape[1] / 2.))

    # Make filter dimension broadcastable and subtract
    centered_X = tf.sub(X, convout)

    # Boolean marks whether or not to perform divisive step
    if use_divisor:
        # Note that the local variances can be computed by using the centered_X
        # tensor. If we convolve this with the mean filter, that should give us
        # the variance at each point. We simply take the square root to get our
        # denominator

        # Compute variances
        sum_sqr_XX = tf.nn.conv2d(tf.square(centered_X), filters, [1,1,1,1], 'SAME')

        # Take square root to get local standard deviation
        denom = tf.sqrt(sum_sqr_XX)

        per_img_mean = tf.reduce_mean(denom)
        divisor = tf.maximum(per_img_mean, denom)
        # Divisise step
        new_X = tf.truediv(centered_X, tf.maximum(divisor, threshold))
    else:
        new_X = centered_X

    return new_X

In [7]:
def gaussian_filter(kernel_shape):
    x = np.zeros(kernel_shape, dtype = float)
    mid = np.floor(kernel_shape[0] / 2.)
    
    for kernel_idx in xrange(0, kernel_shape[2]):
        for i in xrange(0, kernel_shape[0]):
            for j in xrange(0, kernel_shape[1]):
                x[i, j, kernel_idx, 0] = gauss(i - mid, j - mid)
    
    return tf.convert_to_tensor(x / np.sum(x), dtype=tf.float32)

In [8]:
def gauss(x, y, sigma=3.0):
    Z = 2 * np.pi * sigma ** 2
    return  1. / Z * np.exp(-(x ** 2 + y ** 2) / (2. * sigma ** 2))

In [9]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == labels) / predictions.shape[0])

In [10]:
batch_size = 64
patch_size = 5
depth1 = 16
depth2 = 32
depth3 = 64
num_hidden1 = 64
num_hidden2 = 16
shape=[batch_size, image_size, image_size, num_channels]

# Construct a 7-layer CNN.
# C1: convolutional layer, batch_size x 28 x 28 x 16, convolution size: 5 x 5 x 1 x 16
# S2: sub-sampling layer, batch_size x 14 x 14 x 16
# C3: convolutional layer, batch_size x 10 x 10 x 32, convolution size: 5 x 5 x 16 x 32
# S4: sub-sampling layer, batch_size x 5 x 5 x 32
# C5: convolutional layer, batch_size x 1 x 1 x 64, convolution size: 5 x 5 x 32 x 64
# Dropout
# F6: fully-connected layer, weight size: 64 x 16
# Output layer, weight size: 16 x 10

graph = tf.Graph()

with graph.as_default():
    # Input data.
    tf_train_data = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.int64, shape=(batch_size))
    tf_valid_data = tf.constant(valid_data)
    tf_test_data = tf.constant(test_data)

    # Variables.
    conv1_weights = tf.Variable(tf.truncated_normal([5, 5, num_channels, 32], stddev=0.1, dtype=tf.float32))
    conv1_biases = tf.Variable(tf.zeros([32], dtype=tf.float32))
    
    conv2_weights = tf.Variable(tf.truncated_normal([5, 5, 32, 64], stddev=0.1, dtype=tf.float32))
    conv2_biases = tf.Variable(tf.constant(0.1, shape=[64], dtype=tf.float32))
    
    fc1_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * 64, 512], stddev=0.1, dtype=tf.float32))
    fc1_biases = tf.Variable(tf.constant(0.1, shape=[512], dtype=tf.float32))

    fc2_weights = tf.Variable(tf.truncated_normal([512, num_labels], stddev=0.1, dtype=tf.float32))
    fc2_biases = tf.Variable(tf.constant(0.1, shape=[num_labels], dtype=tf.float32))
  
    # Model.
    def model(data, keep_prob, shape):
        LCN = LecunLCN(data, shape)
        
        conv = tf.nn.conv2d(LCN, conv1_weights, strides=[1, 1, 1, 1], padding='SAME')
        relu = tf.nn.relu(tf.nn.bias_add(conv, conv1_biases))
        normalization = tf.nn.local_response_normalization(relu)
        pool = tf.nn.max_pool(normalization, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        
        conv = tf.nn.conv2d(pool, conv2_weights, strides=[1, 1, 1, 1], padding='SAME')
        relu = tf.nn.relu(tf.nn.bias_add(conv, conv2_biases))
        normalization = tf.nn.local_response_normalization(relu)
        pool = tf.nn.max_pool(normalization, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        
        shape = pool.get_shape().as_list()
        reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
        print("Shapes:", reshape.get_shape(), fc1_weights.get_shape(), fc1_biases.get_shape())
        hidden = tf.nn.relu(tf.matmul(reshape, fc1_weights) + fc1_biases)
        return tf.matmul(hidden, fc2_weights) + fc2_biases
  
    # Training computation.
    logits = model(tf_train_data, 0.9375, shape)
    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    # Optimizer.
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(0.05, global_step, 10000, 0.95)
    optimizer = tf.train.AdagradOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(model(tf_train_data, 1.0, shape))
    valid_prediction = tf.nn.softmax(model(tf_valid_data, 1.0, shape))
    test_prediction = tf.nn.softmax(model(tf_test_data, 1.0, shape))

    saver = tf.train.Saver()

Shapes: (64, 4096) (4096, 512) (512,)
Shapes: (64, 4096) (4096, 512) (512,)
Shapes: (6000, 4096) (4096, 512) (512,)
Shapes: (26032, 4096) (4096, 512) (512,)


In [ ]:
num_steps = 10001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        batch_data = train_data[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size)]
        
        feed_dict = {tf_train_data : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)

        if (step % 500 == 0): 
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 5.915077
Minibatch accuracy: 4.7%
Validation accuracy: 10.0%
